# NMT attention based model 
most of the models which were known before this paper were based on the encoder and decoder family they were good but they had some problems 

- they had to encode the whole sentence in just one fixed encoder vector so the decoder had to output the whole target just from this vector so these models strugled with the long sentences 

so in this paper the authors proposed to use attention approch to overcome this drawback so what they had done ? 


In [4]:
import numpy as np
import typing
from typing import Any, Tuple
import tensorflow as tf
import tensorflow_text as tf_text
import matplotlib.pyplot as plt
import re
import matplotlib.ticker as ticker
import string
from string import digits

## we will use english to spanish dataset 

In [5]:
import pathlib
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)
path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

2654208/2638744 [==============================] - 0s 0us/step


## processing the sentences 
here we process the sentence by removing the qoutes and elminating the punctions and add the start and end tocken to the sentence 

In [6]:
def preprocess_sentence(sentence):
    
    num_digits= str.maketrans('','', digits)
    
    sentence= sentence.lower()
    sentence= re.sub(" +", " ", sentence)
    sentence= re.sub("'", '', sentence)
    sentence= sentence.translate(num_digits)
    sentence= re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = sentence.rstrip().strip()
    sentence=  'start_ ' + sentence + ' _end'
    
    return sentence

## load the data 
we load the data and listed it while processing each sentence 

In [7]:
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  inp = [preprocess_sentence(inp) for targ, inp in pairs]
  targ = [preprocess_sentence(targ) for targ, inp in pairs]

  return targ, inp

In [8]:
targ,inp  = load_data(path_to_file)

## process the tockens 
we tokenize the inputs and targets sentences 
and we zero padded them to align the whole inputs and targets 

In [9]:
input_sentence_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='')
# Fit the source sentences to the source tokenizer
input_sentence_tokenizer.fit_on_texts(inp)
input_tensor = input_sentence_tokenizer.texts_to_sequences(inp)
input_tensor= tf.keras.preprocessing.sequence.pad_sequences(input_tensor,padding='post' )

In [10]:
target_sentence_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='')
# Fit the source sentences to the source tokenizer
target_sentence_tokenizer.fit_on_texts(targ)
target_tensor = target_sentence_tokenizer.texts_to_sequences(targ)
target_tensor= tf.keras.preprocessing.sequence.pad_sequences(target_tensor,padding='post' )

In [11]:
BUFFER_SIZE  = len(inp)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((input_tensor , target_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [12]:
source_batch, target_batch =next(iter(dataset))

In [17]:
source_batch.shape

TensorShape([64, 53])

In [13]:
steps_per_epoch= len(input_tensor)//BATCH_SIZE
embedding_dim=256
units=1024
input_vocab_size= len(input_sentence_tokenizer.word_index)+1
target_vocab_size= len(target_sentence_tokenizer.word_index)+1

## the Encoder 
the Encoder start with embedding layer 
after that we pass the embedded inputs throw GRU layer with return sequence prameter to True to return the wohle sequnce not just the last output vector and return_state prameter to True  

the mehtod initialize_hidden_state just initialize the first hidden state to zeros 


In [14]:
class Encoder(tf.keras.Model) : 
  def __init__(self ,embedding_dim ,units , vocab_size  ,batch_size): 
    self.embedding_dim =embedding_dim
    self.units = units 
    self.vocab_size = vocab_size 
    self.batch_size = batch_size
    super(Encoder ,self).__init__()
    self.encoder_embedding = tf.keras.layers.Embedding(vocab_size , embedding_dim ) 
    self.encode_gru = tf.keras.layers.GRU(units , return_sequences=True ,return_state=True,
                                   recurrent_initializer='glorot_uniform')
  def call(self, input_tokens , state ) : 
    embedding_vectors = self.encoder_embedding(input_tokens) 
    output , state = self.encode_gru(embedding_vectors , initial_state = state ) 
    return output , state
  def initialize_hidden_state(self  ):
    return  tf.zeros((self.batch_size, self.units))

In [18]:
encoder = Encoder( embedding_dim, units, input_vocab_size,BATCH_SIZE)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden= encoder(source_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

## the attention block 
this class explain attnetion in details 

to apply attention we need for each time stamp of the decoder we need context vector and to calculate this context vector we need attention weights and to calculate the attention weights we need the score function so we will explain each one in detail 

### the attention weights  
we know that attention weights we will get tell us how mush attention we could pay to each output of the encoder throw spacific time stamp of the decoder 

attention_weights = softmax(score_function) 

### the score function 
but to initialize this attention weights we have to give her some Intuitions about the last time stamp of the decoder to be able to weight the right parts of the encoder 
socre = dense(tanh(dense(last_time_stamp) + (encoder_outputs) )) 

### the context vector 
the context = sum( attention_weights * encoder_outputs ) 



In [34]:
class BahdanauAttention(tf.keras.layers.Layer): 
  def __init__(self, units ): 
    super(BahdanauAttention , self).__init__()
    self.units = units
    self.encoder_output_dense = tf.keras.layers.Dense(units )
    self.decoder_pre_state_dense = tf.keras.layers.Dense(units)
    self.init_attention_weights_dense  = tf.keras.layers.Dense(1) 
  def call(self , query, values) : 
    score = self.init_attention_weights_dense(tf.nn.tanh(self.encoder_output_dense(values) + self.decoder_pre_state_dense(tf.expand_dims(query , 1))))
    attention_weights  = tf.nn.softmax(score ,axis =1)  
    context = tf.math.reduce_sum(attention_weights * values , axis = 1) 
    return context , attention_weights

In [35]:
attention_layer= BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)
print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 53, 1)


## Decoder 
the decoder includes embedding layer for the target sentences 
and Gru that return sequences and states and attention layer 

### enforcing learning 
we will use this teqniuque to teach the model and this basicly instead of using the ouput word of the decoder as next state input we use the right word from our target batch data 

In [38]:
class Decoder(tf.keras.Model): 
  def __init__(self  , embedding_dim , units,vocab_size , batch_size ) : 
    super(Decoder , self).__init__()
    self.embedding_dim = embedding_dim 
    self.units = units 
    self.batch_size = batch_size
    self.vocab_size = vocab_size
    self.decoder_embedding = tf.keras.layers.Embedding(vocab_size , embedding_dim)
    self.decoder_gru = tf.keras.layers.GRU(units , return_sequences=True ,return_state=True,
                                   recurrent_initializer='glorot_uniform')
        
    self.attention = BahdanauAttention(units)
    self.fc= tf.keras.layers.Dense(vocab_size) 
  def call(self, x, hidden, encoder_output ) : 
    # embedd the inputs 
    x = self.decoder_embedding(x) 
    # return the context and attention weights 
    context , attention_weights = self.attention(hidden , encoder_output) 
    # we concat the context vector with the previous right target word 
    x= tf.concat([tf.expand_dims( context, 1), x], 
                                      axis=-1)
    output , state = self.decoder_gru(x )
    output= tf.reshape(output, (-1, output.shape[2]))
    x= self.fc(output)
    return x , state , attention_weights 


In [41]:
decoder= Decoder( embedding_dim, units,target_vocab_size, BATCH_SIZE)
sample_decoder_output, _, _= decoder(tf.random.uniform((BATCH_SIZE,1)), sample_hidden, sample_output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 13556)


In [43]:
optimizer = tf.keras.optimizers.Adam()

## training 
we will use the gradient tape to control the training process 


In [44]:
def train_step(inp, targ, encoder_hidden): 
  loss = 0 
  with tf.GradientTape() as tape : 
    # get output and the hidden states of the encoder 
    encoder_output , encoder_hidden = encoder(inp , encoder_hidden)
    decoder_hidden = encoder_hidden
    # we will assign first input to [start] tocken 
    decoder_input = tf.expand_dims(
            [target_sentence_tokenizer.word_index['start_']] * BATCH_SIZE, 1)
    # looping throw the lenth of the target words 
    for t in range(1, targ.shape[1]):
          # passing encoder_output to the decoder
          predictions, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_hidden)
          loss += tf.keras.losses.sparse_categorical_crossentropy(targ[:, t], predictions)
          # assign the new input as the right word from out target
          decoder_input = tf.expand_dims(targ[:, t], 1)
  batch_loss = (loss / int(targ.shape[1]))
  # take the gradients
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

In [ ]:
EPOCHS=20
for epoch in range(EPOCHS):
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  # train the model using data in bataches 
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
  if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {}'.format(epoch + 1,
                                                   batch,                                                   
                                         batch_loss.numpy()))
      print('Epoch {} Loss {}'.format(epoch + 1,total_loss / steps_per_epoch))

In [ ]:
max_target_length= max(len(t) for t in  target_tensor)
max_input_length= max(len(t) for t in input_tensor)

## evaluation 
you will predict but now we don,t know the rights target word so we will pass the output of each time stamp as new input 

In [ ]:
def evaluate(sentence):
    attention_plot= np.zeros((max_target_length, max_input_length))
    #preprocess the sentnece
    sentence = preprocess_sentence(sentence)
    
    #convert the sentence to index based on word2index dictionary
    inputs= [input_sentence_tokenizer.word_index[i] for i in sentence.split(' ')]
    
    # pad the sequence 
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_input_length, padding='post')
    
    #conver to tensors
    inputs = tf.convert_to_tensor(inputs)
    
    result= ''
    
    # creating encoder
    hidden = [tf.zeros((1, units))]
    encoder_output, encoder_hidden= encoder(inputs, hidden)
    
    # creating decoder
    decoder_hidden = encoder_hidden
    decoder_input = tf.expand_dims([target_sentence_tokenizer.word_index['start_']], 0)
    
    for t in range(max_target_length):
        predictions, decoder_hidden, attention_weights= decoder(decoder_input, decoder_hidden, encoder_output)
        
        # storing attention weight for plotting it
        attention_weights = tf.reshape(attention_weights, (-1,))
        attention_plot[t] = attention_weights.numpy()
        
        prediction_id= tf.argmax(predictions[0]).numpy()
        result += target_sentence_tokenizer.index_word[prediction_id] + ' '
        
        if target_sentence_tokenizer.index_word[prediction_id] == '_end':
            return result,sentence, attention_plot
        
        # predicted id is fed back to as input to the decoder
        decoder_input = tf.expand_dims([prediction_id], 0)
        
    return result,sentence, attention_plot

In [ ]:
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [ ]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)
  
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
translate(u'we dit it .')

## final word 
after going throw the whole model you should reread it again backword to beter understanding of the ambiguous thinks you faced 